In [25]:
import cv2
import numpy as np
def CRR(frame):
    
    # 參數設定
    ww, hh, rh, r = 640, 400, 0.68, 3
    xx1, yy1, xx2, yy2 = int(ww * 0.4), int(hh * rh), int(ww * 0.6), int(hh * rh)
    #p1, p2, p3, p4 = [r+120, hh - r], [ww - r, hh - r+100], [xx2-80, yy2+20], [xx1+100, yy2-20]
    p1, p2, p3, p4 = [100, hh ], [ww, hh+100], [xx2-100, yy2+10], [xx1+100, yy2-30]
    co = (255, 0, 0)
    
    # 讀取並預處理影像
    img = frame
    img1 = cv2.resize(img, (ww, hh))
    img2 = img1.copy()
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    output = cv2.GaussianBlur(cv2.dilate(img1, kernel), (5, 5), 0)
    output = cv2.erode(output, kernel)
    output = cv2.Canny(output, 150, 200)
    
    # 建立遮罩區域
    mask = np.zeros((hh, ww), dtype="uint8")
    cv2.fillPoly(mask, [np.array([p1, p2, p3, p4])], 255)
    output1 = cv2.bitwise_and(output, mask)
    
    # 霍夫變換找線
    lines = cv2.HoughLinesP(output1, 1, np.pi / 180, 40, None, 15, 50)
    
    
    if lines is not None:
        slopes, intercepts = [], []
        for l in lines[:, 0]:  # 直接展開 numpy 陣列
            x1, y1, x2, y2 = l
            s = (y2 - y1) / (x2 - x1) if x2 - x1 != 0 else 0
            b = y1 - s * x1
            if (s >-0.5 or s < -0.82) or (s <0.5 or s > 0.82):
                slopes.append(s)
                intercepts.append(b)
            #cv2.line(img2, (x1, y1), (x2, y2), (255, 0, 0), 2)
    
        # 計算左右斜率最大值
        left_slopes = [s for s in slopes if s < 0]
        right_slopes = [s for s in slopes if s > 0]
        
        if left_slopes and right_slopes:
            s1, b1 = min(left_slopes), intercepts[slopes.index(min(left_slopes))]
            s2, b2 = max(right_slopes), intercepts[slopes.index(max(right_slopes))]
            
            # 計算中點並繪製輔助線
            y1, y2 = hh - r, int(hh * 0.825)
            x_mid = int(((y1 - b1) / s1 + (y1 - b2) / s2) / 2)
            cv2.line(img2, (x_mid, y1), (x_mid, y1 - 15), co, 2)
    
            # 填充區域
            area = np.array([[[(y1 - b1) / s1, y1], [(y2 - b1) / s1, y2],
                              [(y2 - b2) / s2, y2], [(y1 - b2) / s2, y1]]], dtype=np.int32)
            mask = np.zeros_like(img2)
            cv2.fillPoly(mask, area, (0, 50, 0))
            img2 = cv2.addWeighted(img2, 1.0, mask, 1.0, 0)
    
    # 繪製中心標記
    x, y = int(ww / 2) - 1, hh - r
    #cv2.line(img2, (x, y), (x, y - 12), (255, 0, 0), 2)
    
    for i in range(1, 10):
        cv2.line(img2, (x - i * 15, y), (x - i * 15, y - 3), (0, 255, 0), 2)
        cv2.line(img2, (x + i * 15, y), (x + i * 15, y - 3), (0, 255, 0), 2)
    
    return img2

cap = cv2.VideoCapture("LaneVideo.mp4")
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    ret, frame = cap.read()             # 讀取影片的每一幀
    if not ret:
        break
    #print(type(frame))
    frame = CRR(frame)
    #print(type(frame))
    cv2.imshow('oxxostudio', frame)     # 如果讀取成功，顯示該幀的畫面
    if cv2.waitKey(1) == ord('q'):      # 每一毫秒更新一次，直到按下 q 結束
        break
cap.release()                           # 所有作業都完成後，釋放資源
cv2.destroyAllWindows()                 # 結束所有視窗